# ReGenNexus Core - Security Demo

This notebook demonstrates the enhanced security features of ReGenNexus Core.
It shows how to use ECDH-384 encryption, certificate-based authentication, and policy-based access control.

## Setup

First, let's install ReGenNexus directly from GitHub:

In [ ]:
# Clone the repository first
!git clone https://github.com/ReGenNow/ReGenNexus.git
%cd ReGenNexus
# Install in development mode
!pip install -e .
# Install additional dependencies
!pip install ipywidgets matplotlib numpy nest_asyncio cryptography
# Return to the parent directory
%cd ..

In [ ]:
# Add the repository directory to Python path to ensure modules can be imported
import sys
import os

# Get the current working directory
cwd = os.getcwd()
# Add the ReGenNexus directory to the Python path
repo_path = os.path.join(cwd, 'ReGenNexus')
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)
    print(f"Added {repo_path} to Python path")

# Add the src directory to the Python path
src_path = os.path.join(repo_path, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"Added {src_path} to Python path")

# Verify the paths are accessible
print("Current Python path:")
for p in sys.path[:5]:  # Show first 5 paths
    print(f"  {p}")

In [ ]:
import asyncio
import json
import time
import uuid
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import nest_asyncio
import matplotlib.pyplot as plt
import numpy as np
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

# Apply nest_asyncio to allow asyncio to work in Jupyter
nest_asyncio.apply()

In [ ]:
# Import ReGenNexus Core modules with correct paths
from src.security.security import SecurityManager
from src.security.auth import CertificateManager
from src.security.policy import PolicyManager

In [ ]:
# Create output area for logs
log_output = widgets.Output()
display(HTML("<h2>ReGenNexus Core Security Demo</h2>"))
display(HTML("<h3>Log Output:</h3>"))
display(log_output)

In [ ]:
# Create security components
security_manager = None
cert_manager = None
policy_manager = None

async def setup_demo():
    """Set up the demo environment."""
    global security_manager, cert_manager, policy_manager
    
    with log_output:
        print("Setting up security demo environment...")
        
        # Initialize security components
        security_manager = SecurityManager()
        cert_manager = CertificateManager()
        policy_manager = PolicyManager()
        
        await security_manager.initialize()
        await cert_manager.initialize()
        await policy_manager.initialize()
        
        print("Security components initialized")
        print("Demo environment ready!")

In [ ]:
async def generate_key_pair():
    """Generate an ECDH key pair and display public key."""
    with log_output:
        print("Generating ECDH-384 key pair...")
        
        # Generate private key
        private_key = ec.generate_private_key(ec.SECP384R1())
        
        # Get public key
        public_key = private_key.public_key()
        
        # Serialize keys
        private_bytes = private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        )
        
        public_bytes = public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
        
        print("Key pair generated:")
        print(f"Private key: {private_bytes.decode('utf-8')[:64]}...")
        print(f"Public key: {public_bytes.decode('utf-8')[:64]}...")
        
        return private_key, public_key

In [ ]:
async def demonstrate_key_exchange():
    """Demonstrate ECDH key exchange between two entities."""
    with log_output:
        print("Demonstrating ECDH key exchange...")
        
        # Generate key pairs for two entities
        alice_private, alice_public = await generate_key_pair()
        bob_private, bob_public = await generate_key_pair()
        
        print("\nPerforming key exchange...")
        
        # Alice derives shared key using Bob's public key
        alice_shared_key = alice_private.exchange(ec.ECDH(), bob_public)
        
        # Bob derives shared key using Alice's public key
        bob_shared_key = bob_private.exchange(ec.ECDH(), alice_public)
        
        # Derive final symmetric keys using HKDF
        alice_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=b'handshake data'
        ).derive(alice_shared_key)
        
        bob_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=b'handshake data'
        ).derive(bob_shared_key)
        
        # Check if keys match
        keys_match = alice_key == bob_key
        
        print(f"Alice's derived key: {alice_key.hex()[:16]}...")
        print(f"Bob's derived key: {bob_key.hex()[:16]}...")
        print(f"Keys match: {keys_match}")
        
        return keys_match

In [ ]:
async def create_certificate():
    """Create a self-signed certificate for an entity."""
    with log_output:
        print("Creating self-signed certificate...")
        
        # Generate certificate
        entity_id = f"demo_entity_{uuid.uuid4().hex[:8]}"
        cert_data = await cert_manager.create_self_signed_cert(
            entity_id=entity_id,
            common_name=f"Demo Entity {entity_id}",
            organization="ReGenNexus Demo",
            validity_days=30
        )
        
        print(f"Certificate created for entity: {entity_id}")
        print(f"Certificate data: {cert_data['certificate'][:64]}...")
        
        return entity_id, cert_data

async def verify_certificate(entity_id, cert_data):
    """Verify a certificate."""
    with log_output:
        print(f"Verifying certificate for entity: {entity_id}")
        
        # Verify certificate
        is_valid = await cert_manager.verify_certificate(
            entity_id=entity_id,
            certificate=cert_data['certificate']
        )
        
        print(f"Certificate valid: {is_valid}")
        
        return is_valid

In [ ]:
async def demonstrate_policy_enforcement():
    """Demonstrate policy-based access control."""
    with log_output:
        print("Demonstrating policy-based access control...")
        
        # Create entities
        admin_id = "admin_entity"
        user_id = "user_entity"
        guest_id = "guest_entity"
        
        # Create resource
        resource_id = "protected_resource"
        
        # Define policies
        admin_policy = {
            "effect": "allow",
            "actions": ["read", "write", "delete"],
            "resources": [resource_id],
            "conditions": {}
        }
        
        user_policy = {
            "effect": "allow",
            "actions": ["read", "write"],
            "resources": [resource_id],
            "conditions": {}
        }
        
        guest_policy = {
            "effect": "allow",
            "actions": ["read"],
            "resources": [resource_id],
            "conditions": {}
        }
        
        # Set policies
        await policy_manager.set_policy(admin_id, "admin_policy", admin_policy)
        await policy_manager.set_policy(user_id, "user_policy", user_policy)
        await policy_manager.set_policy(guest_id, "guest_policy", guest_policy)
        
        print("Policies set for all entities")
        
        # Check permissions
        admin_read = await policy_manager.check_permission(admin_id, "read", resource_id)
        admin_write = await policy_manager.check_permission(admin_id, "write", resource_id)
        admin_delete = await policy_manager.check_permission(admin_id, "delete", resource_id)
        
        user_read = await policy_manager.check_permission(user_id, "read", resource_id)
        user_write = await policy_manager.check_permission(user_id, "write", resource_id)
        user_delete = await policy_manager.check_permission(user_id, "delete", resource_id)
        
        guest_read = await policy_manager.check_permission(guest_id, "read", resource_id)
        guest_write = await policy_manager.check_permission(guest_id, "write", resource_id)
        guest_delete = await policy_manager.check_permission(guest_id, "delete", resource_id)
        
        print("\nPermission check results:")
        print(f"Admin - Read: {admin_read}, Write: {admin_write}, Delete: {admin_delete}")
        print(f"User - Read: {user_read}, Write: {user_write}, Delete: {user_delete}")
        print(f"Guest - Read: {guest_read}, Write: {guest_write}, Delete: {guest_delete}")
        
        # Visualize permissions
        fig, ax = plt.subplots(figsize=(10, 6))
        
        entities = ['Admin', 'User', 'Guest']
        actions = ['Read', 'Write', 'Delete']
        
        data = np.array([
            [admin_read, admin_write, admin_delete],
            [user_read, user_write, user_delete],
            [guest_read, guest_write, guest_delete]
        ]).astype(int)
        
        im = ax.imshow(data, cmap='YlGn')
        
        # Show all ticks and label them
        ax.set_xticks(np.arange(len(actions)))
        ax.set_yticks(np.arange(len(entities)))
        ax.set_xticklabels(actions)
        ax.set_yticklabels(entities)
        
        # Loop over data dimensions and create text annotations
        for i in range(len(entities)):
            for j in range(len(actions)):
                text = ax.text(j, i, "✓" if data[i, j] else "✗",
                              ha="center", va="center", color="black", fontsize=14)
        
        ax.set_title("Policy-Based Access Control")
        fig.tight_layout()
        plt.show()
        
        return data

In [ ]:
async def cleanup_demo():
    """Clean up the demo environment."""
    global security_manager, cert_manager, policy_manager
    
    with log_output:
        print("Cleaning up security demo environment...")
        
        # Shut down security components
        if security_manager:
            await security_manager.shutdown()
        
        if cert_manager:
            await cert_manager.shutdown()
            
        if policy_manager:
            await policy_manager.shutdown()
            
        print("Security components shut down")
        print("Demo environment cleaned up!")

In [ ]:
# Run setup
asyncio.run(setup_demo())

In [ ]:
# Create UI controls
key_exchange_button = widgets.Button(description="Demo Key Exchange")
certificate_button = widgets.Button(description="Demo Certificates")
policy_button = widgets.Button(description="Demo Policies")
clear_button = widgets.Button(description="Clear Log")
cleanup_button = widgets.Button(description="Cleanup Demo")

# Define button callbacks
def on_key_exchange_button_clicked(b):
    asyncio.ensure_future(demonstrate_key_exchange())

def on_certificate_button_clicked(b):
    async def cert_demo():
        entity_id, cert_data = await create_certificate()
        await verify_certificate(entity_id, cert_data)
    asyncio.ensure_future(cert_demo())

def on_policy_button_clicked(b):
    asyncio.ensure_future(demonstrate_policy_enforcement())

def on_clear_button_clicked(b):
    log_output.clear_output()

def on_cleanup_button_clicked(b):
    asyncio.ensure_future(cleanup_demo())

# Connect callbacks
key_exchange_button.on_click(on_key_exchange_button_clicked)
certificate_button.on_click(on_certificate_button_clicked)
policy_button.on_click(on_policy_button_clicked)
clear_button.on_click(on_clear_button_clicked)
cleanup_button.on_click(on_cleanup_button_clicked)

# Display UI
display(widgets.HBox([
    key_exchange_button, certificate_button, policy_button, clear_button, cleanup_button
]))

## Instructions

1. Click "Demo Key Exchange" to see ECDH-384 key exchange in action
2. Click "Demo Certificates" to create and verify certificates
3. Click "Demo Policies" to see policy-based access control
4. Click "Clear Log" to clear the log output
5. Click "Cleanup Demo" when finished to release resources

This demo shows the enhanced security features of ReGenNexus Core:
- ECDH-384 for secure key exchange
- Certificate-based authentication
- Policy-based access control for fine-grained permissions